### <h2 style="font-size: 40px;">Влияние новостей на стоимость акций</h2>

#### Техническое задание 

Разработать модель для анализа сентимента новостей, связанных с компанией «Русагро», и оценить их влияние на динамику стоимости акций.

#### Описание данных

* **`ragr_news.xlsx`** - новости из различных источников с 1 января 2025 по 30 апреля, которые содержат любое упоминание о компании.
    - `story_id` - уникальный индентификатор новости
    - `story_date` - дата появления новости в 12-часовом формате
    - `title` - заголовок новости
    - `text` - содержание новости
    - `language` - отечественный или иностранный ресурс (ru/en)
    - `source` - ссылка на ресурс новости
    - `url` - ссылка на новость
    - `char_length` - количество символов в новости
    - `token_count` - количество слов в новости


* **`ragr_prices.xlsx`** - файл с дневными ценами закрытия по ценам акции Русагро (с 17 февраля 2025 года).
    - `TRADEDATE` - дата торгов
    - `RGBI` - доходность гос. облигаций
    - `IMOEXF` - динамика индекса рынка акций MOEX
    - `RAGR` - цена закрытия акции
    - `Urals` - стоимость нефти марки Юралз

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
import nltk
import wordninja
import re

# Собственный русскоязычный словарь словарь 
wordninja.DEFAULT_LANGUAGE_MODEL = wordninja.LanguageModel('./russian_dictionary/ru_lang_utf8.txt.gz')
wordninja._SPLIT_RE = re.compile(r'\s+')

### Загрузка данных

In [2]:
# Функция для быстрого ознакомления и парсинга таблицы
def parse_dataset(path):
    df = pd.read_excel(path)
    display(df.head())

    display(round(df.isna().mean() * 100, 2).sort_values(ascending=False).to_frame(name='Процент пропусков'))

    df.info()
    return df


# Функция для проверки наличия файла
def check_dataset(path):    
    if os.path.exists(path):
        return parse_dataset(path)
    else:
        print('Неправильный путь к файлу')

In [3]:
data_news = check_dataset('./datasets/ragr_news.xlsx')

,Unnamed: 0,story_id,story_date,title,text,language,source,url,char_length,token_count
0,4661,5358,1/1/2025 12:00:00 AM,Ц е н т р о б а н к Р Ф з а к р ы л д о ...,Ц е н т р о б а н к Р Ф з а к р ы л д о с т у ...,ru,https://m.interfax.ru/news/,https://m.interfax.ru/1001431,1375,315
1,15249,17527,1/3/2025 12:00:00 AM,В 2024 году Саратовская область зафиксировала ...,В 2024 году Саратовская область зафиксировала ...,ru,https://www.kommersant.ru/regions/64,https://www.kommersant.ru/doc/7421226,1200,247
2,46576,53356,1/9/2025 12:00:00 AM,Губернатор Вячеслав Гладков вручил Премию имен...,Губернатор Вячеслав Гладков вручил Премию имен...,ru,https://bel.aif.ru/,https://bel.aif.ru/society/gubernator-gladkov-...,3146,810
3,49578,57182,1/10/2025 12:00:00 AM,В П р и м о р ь е п о с е в н ы е п л о ...,В П р и м о р ь е п о с е в н ы е п л о щ а д ...,ru,https://tass.ru/ekonomika,https://tass.ru/ekonomika/22847513,2869,652
4,105012,120289,1/17/2025 12:00:00 AM,17.01.2025 10:31 В аграрном секторе России наз...,17.01.2025 10:31 В аграрном секторе России наз...,ru,https://ru24.net/moscow/rss/,https://ru24.net/moscow/395923374/,2513,529


,Процент пропусков
Unnamed: 0,0.0
story_id,0.0
story_date,0.0
title,0.0
text,0.0
language,0.0
source,0.0
url,0.0
char_length,0.0
token_count,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 529 entries, 0 to 528
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   529 non-null    int64 
 1   story_id     529 non-null    int64 
 2   story_date   529 non-null    object
 3   title        529 non-null    object
 4   text         529 non-null    object
 5   language     529 non-null    object
 6   source       529 non-null    object
 7   url          529 non-null    object
 8   char_length  529 non-null    int64 
 9   token_count  529 non-null    int64 
dtypes: int64(4), object(6)
memory usage: 41.5+ KB


Датасет **`data_news`** не содержит пропусков. Дату появления новости `story_date` необходимо привести в 24-часовой формат. Название и содержание новости требуют предобработки. Содержание новости `text` содержит заголовок новости и саму новость.

In [4]:
data_prices = check_dataset('./datasets/ragr_prices.xlsx')

,TRADEDATE,RGBI,IMOEXF,RAGR,Urals
0,2025-02-17,16.62,3319.5,216.34,60.395
1,2025-02-18,16.60,3264.0,210.50,61.015
2,2025-02-19,16.58,3285.0,221.92,62.110
3,2025-02-20,16.41,3298.0,219.52,62.550
4,2025-02-21,16.37,3295.0,219.02,60.500


,Процент пропусков
RGBI,4.05
TRADEDATE,0.00
IMOEXF,0.00
RAGR,0.00
Urals,0.00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   TRADEDATE  74 non-null     datetime64[ns]
 1   RGBI       71 non-null     float64       
 2   IMOEXF     74 non-null     float64       
 3   RAGR       74 non-null     float64       
 4   Urals      74 non-null     float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 3.0 KB


Датасет **`data_prices`** содержит 4% пропусков в признаке `RGBI`. По возможности, следует добавить данные с 1 января 2025 года.

#### Вывод загрузки данных

Датасеты связаны по дате. Данные в таблице **`data_news`** собраны за период *01.01.2025 - 25.04.2025*, в **`data_prices`** за *17.02.2025 - 29.05.2025*. \
В датасете **`data_prices`** содержатся пропуски.

### Предобработка данных

In [5]:
data_news['story_date'] = pd.to_datetime(data_news['story_date'], format='%m/%d/%Y %I:%M:%S %p')

Дата `story_date` из **`data_news`** была переведена в формат 'YYYY-MM-DD'.

In [6]:
data_news['language'].unique()

array(['ru', 'en'], dtype=object)

In [7]:
# Функция для замены обрезанных по буквам слов
def fix_spaced_words(text):
    spaced_word_pattern = r'(?:[А-Яа-яЁё]\s){4,}[А-Яа-яЁё]'

    def replace_match(match):
        spaced = match.group(0)
        glued = spaced.replace(' ', '')
        split = ' '.join(wordninja.split(glued))
        return split

    return re.sub(spaced_word_pattern, replace_match, text)

In [8]:
data_news['title'] = data_news['title'].apply(lambda x: fix_spaced_words(x))
data_news['text'] = data_news['text'].apply(lambda x: fix_spaced_words(x))

Текст и заголовок новостей, имеющий нарезанные буквы (пример: 'П е р е ш е д ш и е н а п р я м о е в л а д е н и е') был предобработан, используя библиотеку wordninja с адаптацией под кириллицу. Данная реализация плохо восстанавливает названия компаний и аббревиатуры даже при их наличии в словаре.